Скрипт для поиска одинаковых или похожих данных для редактора CadEditor. Используется для поиска смещения массивов данных в версиях ромов для разных регионов.

Пример использования скрипта с ромов Tom & Jerry (and Tuffy), версии Japan и USA.

In [1]:
#init python.net & load CadEditor
import sys
cadEditorDir = "d:/DEV/MYGIT/CadEditor/" #test at CadEditor v3.6
sys.path.append(cadEditorDir)

import os
os.chdir(cadEditorDir)

import clr
clr.AddReference("CadEditor")
clr.AddReference("System")
clr.AddReference("System.Windows")
clr.AddReference("CSScriptLibrary.v3.5")

from CadEditor import Globals, ConfigScript, Utils, OffsetRec, MapViewType, MapEditor
from CSScriptLibrary import AsmHelper, CSScript

Загрузка и считывание данных из C#-скрипта

In [13]:
def readDictFromFile(fileName):
    asm = AsmHelper(CSScript.Load(fileName))
    data = asm.CreateObject("Data", None);
    ans = {}
    ans["palOffset"] = data.getPalOffset().beginAddr
    ans["videoOffset"] = data.getVideoOffset().beginAddr
    ans["blocksOffset"] = data.getBlocksOffset().beginAddr
    ans["screensOffset"] = data.getScreensOffset().beginAddr
    ans["enemyAddr"] = data.getLevelRecs()[0].objectsBeginAddr
    ans["cheeseAddr"] = data.getCheeseAddr()
    return ans

Тестовое считывание

In [14]:
cfg = cadEditorDir + "settings_tom_and_jerry/Settings_TomAndJerry-1.cs"
readDictFromFile(cfg)

{'blocksOffset': 46069,
 'cheeseAddr': 33060,
 'enemyAddr': 33107,
 'palOffset': 33018,
 'screensOffset': 33967,
 'videoOffset': 192528}

In [15]:
romNameJ = cadEditorDir + "Tom & Jerry (and Tuffy) (J).nes"
romNameU = cadEditorDir + "Tom & Jerry (and Tuffy) (U) [!].nes"

In [25]:
class DataChecker:
    def __init__(self, jdata, udata):
        self.jdata = jdata
        self.udata = udata
        self.SIZE_TO_CHECK = 64
    def findDataAtAddr(self, addr):
        d = self.jdata[addr:addr+self.SIZE_TO_CHECK]
        return self.udata.find(d)
    def checkAndPrint(self, val, addr):
        newAddr = self.findDataAtAddr(addr)
        equal = addr == newAddr
        print "%16s"%val, "%6s"%hex(addr), equal and "==" or "!=", "%6s"%hex(newAddr)

In [26]:
configNameTempl = cadEditorDir + "settings_tom_and_jerry/Settings_TomAndJerry-%d.cs"

with open(romNameJ, "rb")as f:
    jdata = f.read()
with open(romNameU, "rb")as f:
    udata = f.read()
checker = DataChecker(jdata, udata)

for i in xrange(10):
    levelNo = i + 1
    print "LEVEL NO", levelNo
    configName = configNameTempl % levelNo
    dd = readDictFromFile(configName)
    for k,v in dd.iteritems():
        checker.checkAndPrint(k,v)

LEVEL NO 1
       enemyAddr 0x8153 == 0x8153
   screensOffset 0x84af != 0x84b7
    blocksOffset 0xb3f5 != 0xb3fd
     videoOffset 0x2f010 == 0x2f010
      cheeseAddr 0x8124 == 0x8124
       palOffset 0x80fa == 0x80fa
LEVEL NO 2
       enemyAddr 0x8257 != 0x825b
   screensOffset 0xa23c != 0xa244
    blocksOffset 0xb8b5 != 0xb8bd
     videoOffset 0x30010 == 0x30010
      cheeseAddr 0x8232 != 0x8236
       palOffset 0xa20f != 0xa217
LEVEL NO 3
       enemyAddr 0xc169 == 0xc169
   screensOffset 0xc491 == 0xc491
    blocksOffset 0xdcb0 == 0xdcb0
     videoOffset 0x31010 == 0x31010
      cheeseAddr 0xc144 == 0xc144
       palOffset 0xc134 == 0xc134
LEVEL NO 4
       enemyAddr 0xc2a9 == 0xc2a9
   screensOffset 0xe212 == 0xe212
    blocksOffset 0xf462 == 0xf462
     videoOffset 0x32010 == 0x32010
      cheeseAddr 0xc286 == 0xc286
       palOffset 0xc276 == 0xc276
LEVEL NO 5
       enemyAddr 0x10165 == 0x10165
   screensOffset 0x1045c == 0x1045c
    blocksOffset 0x118ec == 0x118ec
     videoOff